In [0]:
%sql
INSERT INTO genealogy.silver_place_canonical
SELECT
  sha2(place_normalised, 256)      AS canonical_place_id,
  MIN(place_display)               AS preferred_name,
  place_normalised,
  MAX(inferred_country)            AS inferred_country,
  'AUTO_SEED_FROM_VARIANT'         AS created_from,
  current_date                     AS created_at,
  TRUE                             AS is_active,
  NULL                             AS merged_into_canonical_id,
  NULL                             AS merge_reason,
  NULL                             AS merge_timestamp
FROM genealogy.silver_place_variant
WHERE place_normalised not in (SELECT place_normalised FROM genealogy.silver_place_canonical)
GROUP BY place_normalised;


In [0]:
%sql
ALTER TABLE genealogy.silver_place_variant
ADD COLUMNS (canonical_place_id STRING);



In [0]:
%sql
MERGE INTO genealogy.silver_place_variant AS v
USING genealogy.silver_place_canonical AS c
ON v.place_normalised = c.place_normalised
WHEN MATCHED THEN
  UPDATE SET v.canonical_place_id = c.canonical_place_id;

In [0]:
%sql
SELECT *
FROM genealogy.silver_place_variant
WHERE canonical_place_id IS NULL;
